In [13]:
import numpy as np
import torch
from torch.utils import data
from torch import nn
import argparse


In [8]:
class TimeSeriesChunkDataset(data.Dataset):
    def __init__(self, x, y, context):
        super(TimeSeriesChunkDataset, self).__init__()
        self.x = x
        self.y = y
        self.context = context
        self.points_per_series = self.x.shape[1] - self.context + 1
        
    def __len__(self):
        return self.x.shape[0] * self.points_per_series

    def __getitem__(self, index):
        index_series = index // self.points_per_series
        index_point = index % self.points_per_series
        return_x = self.x[index_series, index_point:index_point+self.context, :]
        return_y = np.argmax(self.y[index_series, :])
        return return_x, return_y

In [119]:
class Classifier(nn.Module):
    def __init__(self, layers_size, dim_out, dropout=0.3):
        super(Classifier, self).__init__()
        self.layers = []
        self.activs = []
        self.dropouts = []
        self.layers_size = layers_size
        
        for i in range(len(layers_size)-1):
            self.layers.append(nn.Linear(layers_size[i], layers_size[i+1]))
            self.activs.append(nn.ReLU())
            if i < len(layers_size)-2:
                self.dropouts.append(nn.Dropouts(p=dropout))

        self.layers.append(nn.Linear(layers_size[-1], dim_out))
        self.nlayer = len(self.layers)
        self.nactivs = len(self.activs)
        self.ndropouts = len(self.dropouts)

        self.layers = nn.ModuleList(self.layers)
        
    def forward(self, x):
        out = x
        for i in range(self.nlayer):
            out = self.layers[i](out)
            if i < self.nactivs:
                out = self.activs[i](out)
            if i < self.ndropouts:
                out = self.dropouts[i](out)

        return out


In [120]:
class Discriminator(nn.Module):
    '''
    credit: from https://github.com/martinmamql/complex_da
    '''
    def __init__(self, feature_dim, d_out):
        super(Discriminator, self).__init__()
        self.net = nn.Sequential(

            nn.Linear(feature_dim, feature_dim),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Linear(feature_dim, feature_dim),
            nn.LayerNorm(feature_dim),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Linear(feature_dim, feature_dim),
            nn.LayerNorm(feature_dim),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Linear(feature_dim, feature_dim),
            nn.LayerNorm(feature_dim),
            nn.LeakyReLU(0.2, inplace=True),
        ) 
        self.fc = nn.Linear(3200, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        # x: [bs, seq, feature_dim]
        x = self.net(x)
        bs = x.shape[0]
        x = x.reshape(bs, -1)
        out = self.sigmoid(self.fc(x))
        return out

In [111]:
class Generator(nn.Module):
    def __init__(self, **kwargs):
        super(Generator, self).__init__()
        arguments = {'d_model':2, 'nhead':1, 'num_encoder_layers':3, 'num_decoder_layers':3, 'dim_feedforward':1024, 'dropout':0.1, 'activation':'gelu'}
        arguments.update(kwargs)
        self.transformer = nn.Transformer(**arguments)
        
    def forward(self, x):
        out = self.transformer(x)
        return x
        

In [121]:
class SourceDomainClassifier():
    def __init__(self, context, dim_out, **kwargs):
        super(SourceDomainClassifier, self).__init__()
        self.transformation = Generator(**kwargs)
        self.classifier = Classifier(context*2, dim_out, **kwargs)
        
    def forward(self, x):
        x = self.transformation(x)
        x = nn.Module.flatten(x)
        x = self.classifier(x)
        
        return x
        

In [ ]:
def train(model, train_dataloader, lr, n_epochs, device):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.7, patience=3)
    loss_fn = nn.CrossEntropyLoss()
    train_loss_ = []
    train_acc_ = []
    vali_acc_ = []
    
    for epoch in range(1, n_epochs+1):
        train_loss = 0.0
        train_acc = 0.0

        num_data = 0.0
        num_batches = len(train_dataloader)
        for batches, (x_batch, y_batch) in tqdm(enumerate(train_dataloader), total=num_batches):
            model.train()
            num_data += y_batch.shape[0]
            x_batch = x_batch.to(device)
            y_batch = y_batch.long().to(device)
            optimizer.zero_grad()
            preds = model(x_batch)
            loss = loss_fn(preds, y_batch.squeeze_())
            acc = get_accuracy(preds, y_batch, normalize=False)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            train_acc += acc.item()


In [14]:
#local only

class fake_args():
    def __init__(self, data_path, task, batch_size, epochs, lr, context):
        self.data_path = data_path
        self.task = task
        self.batch_size = batch_size
        self.epochs = epochs
        self.lr = lr
        self.context = context
        
args = fake_args('./data_unzip/,', '3A', 10, 10, 1e-3, 10)

In [18]:
if __name__ == "__main__":
    cuda = torch.cuda.is_available()
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    
    args.task = "processed_file_3Av2.pkl" if args.task == "3A" else "processed_file_3E.pkl"
    args.data_path = args.data_path + args.task
    data = np.load(args.data_path, allow_pickle=True)
    train_dataset = TimeSeriesChunkDataset(data['tr_data'], data['tr_data'], args.context)
    train_dataloader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=4)
    
    

    
